<a href="https://colab.research.google.com/github/rmwvamp/ECGNotebooks/blob/main/GSR_FeatureExtractionCode_CLASDataset_Allfiles_SingleCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Drive Mounting



In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
%cd /gdrive/MyDrive/SamsungWork/Participant2Data

/gdrive/MyDrive/SamsungWork/Participant2Data


## Dependencies

In [3]:
%%capture
!pip install wfdb
!pip install opensignalsreader
!pip install pyhrv
!pip install biosppy
!pip install pyhrv

In [4]:
%%capture
import os
import sys
import csv

#median
import statistics
from statistics import median

import os
from glob import glob

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

# physionet data
import wfdb
from wfdb import processing
import cvxopt as cv
import cvxopt.solvers

import scipy.stats as sp

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',100000)
from __future__ import division, print_function

# Functions

## Detect Peak, cvxEDA and feature extraction function

In [5]:

def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):

    """Detect peaks in data based on their amplitude and other features.
    Parameters
    ----------
    x : 1D array_like
        data.
    mph : {None, number}, optional (default = None)
        detect peaks that are greater than minimum peak height.
    mpd : positive integer, optional (default = 1)
        detect peaks that are at least separated by minimum peak distance (in
        number of data).
    threshold : positive number, optional (default = 0)
        detect peaks (valleys) that are greater (smaller) than `threshold`
        in relation to their immediate neighbors.
    edge : {None, 'rising', 'falling', 'both'}, optional (default = 'rising')
        for a flat peak, keep only the rising edge ('rising'), only the
        falling edge ('falling'), both edges ('both'), or don't detect a
        flat peak (None).
    kpsh : bool, optional (default = False)
        keep peaks with same height even if they are closer than `mpd`.
    valley : bool, optional (default = False)
        if True (1), detect valleys (local minima) instead of peaks.
    show : bool, optional (default = False)
        if True (1), plot data in matplotlib figure.
    ax : a matplotlib.axes.Axes instance, optional (default = None).
    Returns
    -------
    ind : 1D array_like
        indeces of the peaks in `x`.
    Notes
    -----
    The detection of valleys instead of peaks is performed internally by simply
    negating the data: `ind_valleys = detect_peaks(-x)`
    
    The function can handle NaN's 
    See this IPython Notebook [1]_.
    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
    Examples
    --------
    >>> from detect_peaks import detect_peaks
    >>> x = np.random.randn(100)
    >>> x[60:81] = np.nan
    >>> # detect all peaks and plot data
    >>> ind = detect_peaks(x, show=True)
    >>> print(ind)
    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # set minimum peak height = 0 and minimum peak distance = 20
    >>> detect_peaks(x, mph=0, mpd=20, show=True)
    >>> x = [0, 1, 0, 2, 0, 3, 0, 2, 0, 1, 0]
    >>> # set minimum peak distance = 2
    >>> detect_peaks(x, mpd=2, show=True)
    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # detection of valleys instead of peaks
    >>> detect_peaks(x, mph=0, mpd=20, valley=True, show=True)
    >>> x = [0, 1, 1, 0, 1, 1, 0]
    >>> # detect both edges
    >>> detect_peaks(x, edge='both', show=True)
    >>> x = [-2, 1, -2, 2, 1, 1, 3, 0]
    >>> # set threshold = 2
    >>> detect_peaks(x, threshold = 2, show=True)
    """

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)

    return ind


In [6]:
def movingAvg(dt):
    N = 16
    cumsum, moving_aves = [0], []
    for i, x in enumerate(dt, 1):
        cumsum.append(cumsum[i-1] + x)
        if i>=N:
            moving_ave = (cumsum[i] - cumsum[i-N])/N
            moving_aves.append(moving_ave)
    return moving_aves

In [7]:
def cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    """CVXEDA Convex optimization approach to electrodermal activity processing
    This function implements the cvxEDA algorithm described in "cvxEDA: a
    Convex Optimization Approach to Electrodermal Activity Processing"
    (http://dx.doi.org/10.1109/TBME.2015.2474131, also available from the
    authors' homepages).
    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see:
                http://cvxopt.org/userguide/coneprog.html#algorithm-parameters
    Returns (see paper for details):
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """

    n = len(y)
    y = cv.matrix(y)

    # bateman ARMA model
    a1 = 1./min(tau1, tau0) # a1 > a0
    a0 = 1./max(tau1, tau0)
    ar = np.array([(a1*delta + 2.) * (a0*delta + 2.), 2.*a1*a0*delta**2 - 8.,
        (a1*delta - 2.) * (a0*delta - 2.)]) / ((a1 - a0) * delta**2)
    ma = np.array([1., 2., 1.])

    # matrices for ARMA model
    i = np.arange(2, n)
    A = cv.spmatrix(np.tile(ar, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))
    M = cv.spmatrix(np.tile(ma, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))

    # spline
    delta_knot_s = int(round(delta_knot / delta))
    spl = np.r_[np.arange(1.,delta_knot_s), np.arange(delta_knot_s, 0., -1.)] # order 1
    spl = np.convolve(spl, spl, 'full')
    spl /= max(spl)
    # matrix of spline regressors
    i = np.c_[np.arange(-(len(spl)//2), (len(spl)+1)//2)] + np.r_[np.arange(0, n, delta_knot_s)]
    nB = i.shape[1]
    j = np.tile(np.arange(nB), (len(spl),1))
    p = np.tile(spl, (nB,1)).T
    valid = (i >= 0) & (i < n)
    B = cv.spmatrix(p[valid], i[valid], j[valid])

    # trend
    C = cv.matrix(np.c_[np.ones(n), np.arange(1., n+1.)/n])
    nC = C.size[1]

    # Solve the problem:
    # .5*(M*q + B*l + C*d - y)^2 + alpha*sum(A,1)*p + .5*gamma*l'*l
    # s.t. A*q >= 0

    old_options = cv.solvers.options.copy()
    cv.solvers.options.clear()
    cv.solvers.options.update(options)
    if solver == 'conelp':
        # Use conelp
        z = lambda m,n: cv.spmatrix([],[],[],(m,n))
        G = cv.sparse([[-A,z(2,n),M,z(nB+2,n)],[z(n+2,nC),C,z(nB+2,nC)],
                    [z(n,1),-1,1,z(n+nB+2,1)],[z(2*n+2,1),-1,1,z(nB,1)],
                    [z(n+2,nB),B,z(2,nB),cv.spmatrix(1.0, range(nB), range(nB))]])
        h = cv.matrix([z(n,1),.5,.5,y,.5,.5,z(nB,1)])
        c = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T,z(nC,1),1,gamma,z(nB,1)])
        res = cv.solvers.conelp(c, G, h, dims={'l':n,'q':[n+2,nB+2],'s':[]})
        obj = res['primal objective']
    else:
        # Use qp
        Mt, Ct, Bt = M.T, C.T, B.T
        H = cv.sparse([[Mt*M, Ct*M, Bt*M], [Mt*C, Ct*C, Bt*C], 
                    [Mt*B, Ct*B, Bt*B+gamma*cv.spmatrix(1.0, range(nB), range(nB))]])
        f = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T - Mt*y,  -(Ct*y), -(Bt*y)])
        res = cv.solvers.qp(H, f, cv.spmatrix(-A.V, A.I, A.J, (n,len(f))),
                            cv.matrix(0., (n,1)), solver=solver)
        obj = res['primal objective'] + .5 * (y.T * y)
    cv.solvers.options.clear()
    cv.solvers.options.update(old_options)

    l = res['x'][-nB:]
    d = res['x'][n:n+nC]
    t = B*l + C*d
    q = res['x'][:n]
    p = A * q
    r = M * q
    e = y - r - t

    return (np.array(a).ravel() for a in (r, p, t, l, d, e, obj))




In [8]:
def process(y,delta):
    [r, p, t, l, d, e, obj] = cvxEDA(y, delta)
    smoothed_phasic=movingAvg(r)
    phasic_peaks=detect_peaks(smoothed_phasic, mph=0.001, mpd=int(1/delta), show=False)
    temp={}
    #Calculate the moments of the GSR data
    temp["mean_gsr"]=np.mean(y)
    temp["var_gsr"]=np.var(y)
    temp["skew_gsr"]=sp.skew(y)
    temp["kurtosis_gsr"]=sp.kurtosis(y)
    temp["std_gsr"]=np.std(y)
    #Calculate the moments of the SCL data (tonic)
    temp["mean_scl"]=np.mean(t)
    temp["var_scl"]=np.var(t)
    temp["skew_scl"]=sp.skew(t)
    temp["kurtosis_scl"]=sp.kurtosis(t)
    temp["std_scl"]=np.std(t)
    temp["slope_scl"]=np.max(t)-np.min(t)
    #Calculate the moments of the SCR data (phasic)
    temp["mean_scr"]=np.mean(r)
    temp["var_scr"]=np.var(r)
    temp["skew_scr"]=sp.skew(r)
    temp["kurtosis_scr"]=sp.kurtosis(r)
    temp["std_scr"]=np.std(r)
    temp["max_scr"]=np.max(r)
    temp["scr_peaks"]=len(phasic_peaks)
    return temp

## Logical Functions

In [9]:
def windowing(df,windowsizerow):
  column_names=['mean_gsr','var_gsr','skew_gsr','kurtosis_gsr','std_gsr','mean_scl','var_scl','skew_scl','kurtosis_scl','std_scl','slope_scl','mean_scr','var_scr','skew_scr','kurtosis_scr','std_scr','max_scr','scr_peaks']
  result_df= pd.DataFrame(columns=column_names) 
  df1=df.rolling(window=windowsizerow)
  type(df1)
  i=1
  j=0
  for df2 in df1:
    if len(df2)>=windowsizerow and i%int(windowsizerow/2)==0:
     results=process(df2.gsr,1./256)
     result_df=result_df.append(results,ignore_index=True)
      # print(result_df)
    i+=1
    j+=1
    
  return result_df

## Function for Feature Extraction

In [10]:
def baseline_stress_features(input_path=r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants/Part4",output_path=r"/gdrive/MyDrive/SamsungWork/ECGFeatures10Seconds_1May22",partnumber="Part4"):
  %cd "$input_path"  
  # File setup for baseline dataframe
  baseline_file=['1_gsr_ppg_.csv','4_gsr_ppg_.csv','7_gsr_ppg_.csv','10_gsr_ppg_.csv']
  baseline_df=pd.read_csv(os.path.join(input_path,baseline_file[0]))
  column_names=['mean_gsr','var_gsr','skew_gsr','kurtosis_gsr','std_gsr','mean_scl','var_scl','skew_scl','kurtosis_scl','std_scl','slope_scl','mean_scr','var_scr','skew_scr','kurtosis_scr','std_scr','max_scr','scr_peaks']

 # File setup for stress dataframe
  stress_df=pd.read_csv('2_gsr_ppg_mathtest.csv')
  data=pd.read_csv('8_gsr_ppg_IQtest.csv')
  stress_df=stress_df.append(data,ignore_index=True)



  # removing unnecessary columns
  baseline_df=baseline_df.drop(['Timestamp','ppg','PythonTimestamp','accelx','accely','accelz'],axis=1)
  stress_df=stress_df.drop(['Timestamp','ppg','PythonTimestamp','accelx','accely','accelz'],axis=1)

  # appending baselinedataframe, adding all data baseline together
  for base in baseline_file[1:]:
    data=pd.read_csv(base)
    baseline_df=baseline_df.append(data,ignore_index=True)

  labelling_criteria=0

  # setting up folders for saving
  if not os.path.exists(os.path.join(output_path,partnumber,"output",str(labelling_criteria))):
    os.makedirs(os.path.join(output_path,partnumber,"output",str(labelling_criteria)))

  # baseline feature extraction
  # print(baseline_df)
  result_baseline_df=windowing(baseline_df,1280)
  result_baseline_df['stress_level']=0;
  result_baseline_df.to_csv(os.path.join(output_path,partnumber,"output",str(labelling_criteria),"baseline.csv"))
  print(result_baseline_df)

   # stress feature extraction
  # %%capture
  # print(stress_df)
  result_stress_df=windowing(stress_df,1280)
  

  result_stress_df['stress_level']=1;
  result_stress_df.to_csv(os.path.join(output_path,partnumber,"output",str(labelling_criteria),"stress.csv"))
  print(result_stress_df)
  # print(partnumber)

  print("Feature Extracted for %s saved at %s " %(partnumber,os.path.join(output_path,partnumber)))

# 5 Seconds Window, 2.5 Overlapping

In [11]:
Participants5=['Part4','Part5','Part6','Part8','Part10','Part11','Part14','Part17','Part24','Part26','Part28','Part31','Part32','Part33','Part34','Part35','Part36','Part39','Part40','Part42','Part43','Part44','Part45','Part46','Part48','Part49','Part51','Part52','Part53','Part54','Part55','Part57','Part58','Part60']

In [12]:
Error_5=['Part1','Part2','Part3','Part7','Part9','Part12','Part13','Part15','Part16','Part18','Part19','Part20','Part21','Part22','Part23','Part25','Part27','Part29','Part30','Part37','Part38','Part41','Part47','Part50','Part56','Part59']

In [13]:
# 29->55

In [14]:
for Part in Participants5[:]:
    baseline_stress_features(os.path.join(r"/gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants", Part,"by_block"),r"/gdrive/MyDrive/SamsungWork/GSReatures_20May22_Final",Part)

Streaming output truncated to the last 5000 lines.
 0: -1.8246e+10 -1.8246e+10  2e+03  5e+01  7e-02
 1: -1.8246e+10 -1.8246e+10  3e+02  9e+00  1e-02
 2: -1.8246e+10 -1.8246e+10  4e+02  8e+00  1e-02
 3: -1.8246e+10 -1.8246e+10  4e+02  8e+00  1e-02
 4: -1.8246e+10 -1.8246e+10  1e+03  4e+00  6e-03
 5: -1.8246e+10 -1.8246e+10  1e+03  3e+00  4e-03
 6: -1.8246e+10 -1.8246e+10  6e+02  1e+00  2e-03
 7: -1.8246e+10 -1.8246e+10  6e+02  9e-01  1e-03
 8: -1.8246e+10 -1.8246e+10  6e+02  8e-01  1e-03
 9: -1.8246e+10 -1.8246e+10  6e+02  6e-01  8e-04
10: -1.8246e+10 -1.8246e+10  6e+02  6e-01  8e-04
11: -1.8246e+10 -1.8246e+10  6e+02  5e-01  7e-04
12: -1.8246e+10 -1.8246e+10  6e+02  3e-01  5e-04
13: -1.8246e+10 -1.8246e+10  6e+02  3e-01  5e-04
14: -1.8246e+10 -1.8246e+10  8e+02  3e-01  4e-04
15: -1.8246e+10 -1.8246e+10  8e+02  2e-01  3e-04
16: -1.8246e+10 -1.8246e+10  8e+02  2e-01  3e-04
17: -1.8246e+10 -1.8246e+10  9e+02  2e-01  3e-04
18: -1.8246e+10 -1.8246e+10  9e+02  2e-01  3e-04
19: -1.8246e+10 -1

# Combining Features

In [15]:
PATH = r"/gdrive/MyDrive/SamsungWork/GSReatures_20May22_Final"
EXT = "*.csv"
all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
# len(all_csv_files)
column_names=['mean_gsr','var_gsr','skew_gsr','kurtosis_gsr','std_gsr','mean_scl','var_scl','skew_scl','kurtosis_scl','std_scl','slope_scl','mean_scr','var_scr','skew_scr','kurtosis_scr','std_scr','max_scr','scr_peaks']

combineddf= pd.DataFrame(columns=column_names) 


In [16]:
for featuredata in all_csv_files:
      tempdata=pd.read_csv(featuredata)
      combineddf=combineddf.append(tempdata,ignore_index=True)


In [17]:
combineddf.to_csv(r"/gdrive/MyDrive/SamsungWork/GSReatures_20May22_Final/CombinedFeatures.csv")


In [18]:
print(combineddf)

        mean_gsr    var_gsr  skew_gsr  kurtosis_gsr   std_gsr    mean_scl  \
0      40.587001   0.775156 -0.258075     -0.901524  0.880429   35.251333   
1      40.117051   1.324220  0.026227     -1.425200  1.150748   34.084713   
2      39.777054   0.682222  0.116371     -0.997151  0.825967   35.408139   
3      40.656780   0.741155 -0.281826     -1.251442  0.860904   38.637834   
4      41.620884   0.275586 -0.182381     -0.908925  0.524963   37.177244   
...          ...        ...       ...           ...       ...         ...   
8203  119.942537   0.013849  0.006937      0.181163  0.117680  119.619422   
8204  118.914486   4.369317 -1.645810      1.236698  2.090291  113.038544   
8205  114.773393  12.746827  0.536207     -1.430533  3.570270  108.677476   
8206  112.636690   1.364320  0.612033     -0.992617  1.168041  112.851369   
8207  114.840383   2.158477 -0.342474     -1.140484  1.469176  114.569412   

        var_scl  skew_scl  kurtosis_scl   std_scl  slope_scl  mean_scr  \
0